In [4]:
import Levenshtein

In [7]:
def Edit_distance_str(str1,str2):
    edit_distance = Levenshtein.distance(str1,str2)
    similarity = 1-(edit_distance/max(len(str1),len(str2)))
    return {'distance':edit_distance,'similarity':similarity}

In [8]:
a = Edit_distance_str('asdfgh','asdfgj')

In [9]:
a

{'distance': 1, 'similarity': 0.8333333333333334}

In [10]:
'''
计算相似度，前提是：将文本转化为向量
使用jieba分词，词袋模型，tf-idf模型，LSI模型 
'''

'\n计算相似度，前提是：将文本转化为向量\n使用jieba分词，词袋模型，tf-idf模型\n'

In [15]:
import jieba.posseg as pseg
import codecs
from gensim import corpora,models,similarities

In [41]:
#构建停用词表
def stopwords_list(filepath):
    stop_words_file = filepath
    stopwords = codecs.open(stop_words_file,'r',encoding='utf8').readlines()
    stopwords_list = [words.strip() for words in stopwords]
    stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
    return stopwords_list,stop_flag

#对一篇文章，或者句子分词、去停用词
def tokenization(filename):
    res = []
    with open(filename,'r') as f:
        text = f.read()
        words = pseg.cut(text)
    stopwordslist,stop_flag = stopwords_list('/Users/taoxudong/huli/test/stop_words.txt')
    #分词特性？中文对应着stop_flag的含义。？？？？？？
    for word,flag in words:
        if word not in stopwordslist and flag not in stop_flag:
            if word != '\t':
                res.append(word)
    return res

#加载要处理的文本，进行分词
filenames = ['/Users/taoxudong/huli/test/高血压.txt',
            '/Users/taoxudong/huli/test/低血压.txt',
            '/Users/taoxudong/huli/test/篮球.txt']

corpus = []
for each in filenames:
    corpus.append(tokenization(each))
#建立词袋模型，生成字典和向量语料
print(corpus)
dictionary = corpora.Dictionary(corpus)
doc_vectors = [dictionary.doc2bow(text) for text in corpus]

<generator object cut at 0x1a1fbdd048>
<generator object cut at 0x1a1fbdd0c0>
<generator object cut at 0x1a1fbdd0c0>
[['高血压', 'hypertension', '是', '指以', '体循环', '动脉', '血压', '收缩压', '舒张压', '增高', '主要', '特征', '收缩压', '毫米汞柱', '舒张压', '毫米汞柱', '可', '伴有', '心', '脑', '肾', '器官', '功能', '器质性', '损害', '临床', '综合征', '高血压', '是', '常见', '慢性病', '是', '心脑血管病', '主要', '危险', '因素', '正常人', '血压', '随', '内外', '环境', '变化', '范围', '内', '波动', '整体', '人群', '血压', '水平', '随', '年龄', '升高', '收缩压', '明显', '舒张压', '呈现', '下降', '趋势', '脉压', '加大', '人们', '心血管病', '危险', '因素', '作用', '心', '脑', '肾', '器官', '保护', '认识', '不断深入', '高血压', '诊断', '标准', '调整', '认为', '同一', '血压', '水平', '患者', '发生', '心血管病', '危险', '不同', '有', '了', '血压', '分层', '概念', '即', '发生', '心血管病', '危险度', '不同', '患者', '适宜', '血压', '水平', '应有', '不同', '血压', '值', '危险', '因素', '评估', '是', '诊断', '制定', '高血压', '治疗', '方案', '主要', '不同', '患者', '高血压', '管理', '目标', '不同', '医生', '面对', '患者', '时', '参考', '标准', '基础', '具体情况', '判断', '患者', '最合适', '血压', '范围', '采用', '针对性', '治疗', '措施', '改善生活', '方式', '基础', '推荐', '使用', '小时', 

In [38]:
#建立tf-idf模型
tfidf = models.TfidfModel(doc_vectors)
tfidf_vectors = tfidf[doc_vectors]
#len(tfidf_vectors) = 文本数  len(tfidf_vectors[0] == 第一个文本中的词数量)
print(len(tfidf_vectors))
query = tokenization('/Users/taoxudong/huli/test/血压类的.txt')
print(query)
query_bow = dictionary.doc2bow(query)
index = similarities.MatrixSimilarity(tfidf_vectors)
sims = index[query_bow]
print(list(enumerate(sims)))

3
['一般来说', '高血压', '属于', '慢性病', '患者', '需要', '药物', '治疗', '有效', '控制', '血压', '到', '目标', '水平', '一般来说', '要求', '血压', '低于', 'mmHg', '减少', '心脑血管', '事件', '如', '防止', '卒', '冠心病', '心力衰竭', '肾病', '发生', '进展', '有效', '降低', '死亡', '风险']
[(0, 0.5735908), (1, 0.054820858), (2, 0.0)]


In [39]:
#构建LSI模型，设置主题数（类数）
lsi = models.LsiModel(tfidf_vectors,id2word = dictionary,num_topics=2)
lsi_vector = lsi[tfidf_vectors]
#LSI向量空间中，所有文本的向量都是二维的
query = tokenization('/Users/taoxudong/huli/test/血压类的.txt')
query_bow = dictionary.doc2bow(query)
query_lsi = lsi[query_bow]
print(query)
index = similarities.MatrixSimilarity(lsi_vector)
sims = index[query_lsi]
print(list(enumerate(sims)))

['一般来说', '高血压', '属于', '慢性病', '患者', '需要', '药物', '治疗', '有效', '控制', '血压', '到', '目标', '水平', '一般来说', '要求', '血压', '低于', 'mmHg', '减少', '心脑血管', '事件', '如', '防止', '卒', '冠心病', '心力衰竭', '肾病', '发生', '进展', '有效', '降低', '死亡', '风险']
[(0, 0.9999886), (1, 0.998825), (2, 0.022608727)]
